In [1]:
import json
from collections import defaultdict
from sqlalchemy import create_engine
from django.core.management.base import BaseCommand
from django.conf import settings
from appcreator.import_utils import run_import

# imports for custom things
import json
from tqdm import tqdm
import pandas as pd
from django.core.serializers.json import DjangoJSONEncoder
from archiv.models import KeyWord, Stelle


dbc = settings.LEGACY_DB_CONNECTION
db_connection_str = f"mysql+pymysql://{dbc['USER']}:{dbc['PASSWORD']}@{dbc['HOST']}/{dbc['NAME']}"
db_connection = create_engine(db_connection_str)

In [2]:
for x in KeyWord.objects.all():
    x.stichwort = x.stichwort.strip()
    x.save()

In [3]:
query = "SELECT * FROM stelle"
df = pd.read_sql(query, con=db_connection)

In [4]:
stichwort_list = [x for x in list(df.keys()) if x.startswith('sstich')]
ambique = {}
no_match = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    if row['sid']:
        try:
            item = Stelle.objects.get(legacy_pk=row['sid'])
        except:
            continue
        for st in stichwort_list:
            if row[st] != "":
                try:
                    lookup = row[st].strip()
                except AttributeError:
                    continue
                kws = KeyWord.objects.filter(stichwort=lookup)
                if len(kws) == 1:
                    item.key_word.add(*kws)
                elif len(kws) == 0:
                    no_match[f"{item}"] = lookup
                else:
                    ambique[f"{item}"] = lookup

100%|██████████| 4661/4661 [00:49<00:00, 94.20it/s] 


In [5]:
print(f"no_match: {len(no_match)}; ambique: {len(ambique)}")

no_match: 1231; ambique: 0


In [6]:
v = defaultdict(list)

In [7]:
for key, value in sorted(no_match.items()):
    v[value].append(key)

In [8]:
with open('no_match.json', 'w') as fp:
    json.dump(v, fp, indent=True)

In [36]:
with open('no_match.json') as fp:
    no_match = json.load(fp)

In [46]:
no_match_in_variants = []
matches = defaultdict(list)
ambique = defaultdict(list)
for key, value in v.items():
    potential_matches = KeyWord.objects.filter(varianten__icontains=key)
    if potential_matches.count() == 0:
        no_match_in_variants.append({key: value})
    elif potential_matches.count() == 1:
        matches[key] = potential_matches
    else:
        ambique[key] = potential_matches

In [47]:
results = [
    ["no_match_in_variants", no_match_in_variants],
    ["matches", matches],
    ["ambique", ambique]
]

In [48]:
for x in results:
    print(f"{x[0]}: {len(x[1])}")

no_match_in_variants: 37
matches: 92
ambique: 16


In [49]:
for item in ambique.items():
    print(f"keyword: {item[0]}\n, varianten: {[(x, x.varianten) for x in item[1]]} \n\n")

keyword: Franci
, varianten: [(<KeyWord: Francia>, 'Franciae;Franciam'), (<KeyWord: francicus>, 'francic'), (<KeyWord: Francigenus>, 'Francigena;Francigeno;Francigeni;Francigenos;Francigenis'), (<KeyWord: Francus>, 'Franci;Francos;Francis')] 


keyword: Graeci
, varianten: [(<KeyWord: Graecia>, 'Graeciae;Graeciam'), (<KeyWord: Graecus>, 'Grecus;Graeci;Greci;Graec')] 


keyword: Gothus
, varianten: [(<KeyWord: Gothi>, 'Gothus;Gothorum;Gothos'), (<KeyWord: Wisigothi>, 'Wisigothus;Wisigotha;Wisigothae')] 


keyword: Romani
, varianten: [(<KeyWord: civis Romanus>, 'civem Romanum;civi Romano;civis Romani;cives Romani;civibus Romanis'), (<KeyWord: Romania>, 'Romaniae;Romaniam'), (<KeyWord: Romanus>, 'Romani;Romanicus;Romana;Roman')] 


keyword: gentes
, varianten: [(<KeyWord: gens>, 'gentes;genti;gente'), (<KeyWord: gens Dei>, 'gentis Dei;genti Dei;gentem Dei;gente Dei;gentes Dei')] 


keyword: Alamanni
, varianten: [(<KeyWord: Alamannia>, 'Alamanniae;Alamanniam;Alemannia'), (<KeyWord: Alama

In [50]:
ambique, [list(x.keys())[0] for x in no_match_in_variants]

(defaultdict(list,
             {'Franci': <QuerySet [<KeyWord: Francia>, <KeyWord: francicus>, <KeyWord: Francigenus>, <KeyWord: Francus>]>,
              'Graeci': <QuerySet [<KeyWord: Graecia>, <KeyWord: Graecus>]>,
              'Gothus': <QuerySet [<KeyWord: Gothi>, <KeyWord: Wisigothi>]>,
              'Romani': <QuerySet [<KeyWord: civis Romanus>, <KeyWord: Romania>, <KeyWord: Romanus>]>,
              'gentes': <QuerySet [<KeyWord: gens>, <KeyWord: gens Dei>]>,
              'Alamanni': <QuerySet [<KeyWord: Alamannia>, <KeyWord: Alamannus>]>,
              'Scotti': <QuerySet [<KeyWord: Scottia>, <KeyWord: Scotticus>, <KeyWord: Scottus>]>,
              'populi': <QuerySet [<KeyWord: gens populi>, <KeyWord: populi gentium>, <KeyWord: populi nationum>, <KeyWord: populus>, <KeyWord: populus christianus>, <KeyWord: populus Dei>]>,
              'Aquitani': <QuerySet [<KeyWord: Aquitania>, <KeyWord: Aquitanus>]>,
              'Hispani': <QuerySet [<KeyWord: Hispania>, <KeyWord: Hi

In [51]:
no_match_in_variants = []
matches = defaultdict(list)
ambique = defaultdict(list)
for key, value in v.items():
    potential_matches = KeyWord.objects.filter(varianten__contains=key)
    if potential_matches.count() == 0:
        no_match_in_variants.append({key: value})
    elif potential_matches.count() == 1:
        matches[key] = potential_matches
    else:
        ambique[key] = potential_matches

In [52]:
results = [
    ["no_match_in_variants", no_match_in_variants],
    ["matches", matches],
    ["ambique", ambique]
]

In [53]:
for x in results:
    print(f"{x[0]}: {len(x[1])}")

no_match_in_variants: 48
matches: 83
ambique: 14


In [54]:
ambique, [list(x.keys())[0] for x in no_match_in_variants]

(defaultdict(list,
             {'Franci': <QuerySet [<KeyWord: Francia>, <KeyWord: Francigenus>, <KeyWord: Francus>]>,
              'Graeci': <QuerySet [<KeyWord: Graecia>, <KeyWord: Graecus>]>,
              'Romani': <QuerySet [<KeyWord: civis Romanus>, <KeyWord: Romania>, <KeyWord: Romanus>]>,
              'gentes': <QuerySet [<KeyWord: gens>, <KeyWord: gens Dei>]>,
              'Alamanni': <QuerySet [<KeyWord: Alamannia>, <KeyWord: Alamannus>]>,
              'Scotti': <QuerySet [<KeyWord: Scottia>, <KeyWord: Scotticus>, <KeyWord: Scottus>]>,
              'populi': <QuerySet [<KeyWord: gens populi>, <KeyWord: populi gentium>, <KeyWord: populi nationum>, <KeyWord: populus>, <KeyWord: populus christianus>, <KeyWord: populus Dei>]>,
              'Aquitani': <QuerySet [<KeyWord: Aquitania>, <KeyWord: Aquitanus>]>,
              'Hispani': <QuerySet [<KeyWord: Hispania>, <KeyWord: Hispaniensis>, <KeyWord: Hispanus>]>,
              'Frisia': <QuerySet [<KeyWord: Fresia>, <KeyWord:

In [55]:
for item in ambique.items():
    print(f"keyword: {item[0]}\n, varianten: {[(x, x.varianten) for x in item[1]]} \n\n")

keyword: Franci
, varianten: [(<KeyWord: Francia>, 'Franciae;Franciam'), (<KeyWord: Francigenus>, 'Francigena;Francigeno;Francigeni;Francigenos;Francigenis'), (<KeyWord: Francus>, 'Franci;Francos;Francis')] 


keyword: Graeci
, varianten: [(<KeyWord: Graecia>, 'Graeciae;Graeciam'), (<KeyWord: Graecus>, 'Grecus;Graeci;Greci;Graec')] 


keyword: Romani
, varianten: [(<KeyWord: civis Romanus>, 'civem Romanum;civi Romano;civis Romani;cives Romani;civibus Romanis'), (<KeyWord: Romania>, 'Romaniae;Romaniam'), (<KeyWord: Romanus>, 'Romani;Romanicus;Romana;Roman')] 


keyword: gentes
, varianten: [(<KeyWord: gens>, 'gentes;genti;gente'), (<KeyWord: gens Dei>, 'gentis Dei;genti Dei;gentem Dei;gente Dei;gentes Dei')] 


keyword: Alamanni
, varianten: [(<KeyWord: Alamannia>, 'Alamanniae;Alamanniam;Alemannia'), (<KeyWord: Alamannus>, 'Alamanni;Alamannorum;Alamannis;Alamannos;Alamanno')] 


keyword: Scotti
, varianten: [(<KeyWord: Scottia>, 'Scottiae;Scotia;Scottiam'), (<KeyWord: Scotticus>, 'Scoti